## Intro

In this PA, you will practice altering data frame objects in order to solve a riddle.

### Advice for troubleshooting your code

#### Check results often

If a chunk of code runs smoothly without giving you any error or warnings this does not necessarily mean it accomplished the desired task.

It is a good habit to check the results of your code every time you finish a task.

If you created or updated a data frame, make sure your edits did what you hoped.  Use the *Environment* or the `head()` function to investigate your changes.

#### Document your work

In the text before the code chunk, make sure to briefly state what the point of the chunk is.  This will remind later readers - which might be your future self! - what the desired output is.

Use comments in the code to give quick explanations of the goals of complex steps.

#### Two heads are better than one

It can be hard to spot bugs in code that you yourself wrote.t
(Have you ever sent a text message and not spotted an obvious  typo?  Same idea!)

Work with people around you if you can - if something goes wrong, ask a friend to take a peek at your code and see if any glaring errors (like *syntax error*) pop out.

#### Explain your code out loud

The best way to troubleshoot a sneaky bug is to explain out loud each step of your code, and what you hoped to accomplish.

If you are alone, try [Rubber Duck Debugging](https://en.wikipedia.org/wiki/Rubber_duck_debugging)!

#### Google is your friend

The whole of the internet is at your disposal!  Use it early, use it often.

Copy-paste the exact error message into Google. Chances are, somebody else had a similar problem and got a similar message.


Include package names in your search terms.  For example, "bar plot in plotnine with pandas data frame" is a better search than "bar plot in python".


## Part One: Data import and cleaning

This section will walk you through some steps to clean up today's dataset, so that you can use it more easily in Part Two.  

First, we declare our package dependencies and load the data.

(Note that the data loading function `read_csv` will give you an outpouring of helpful information about the dataset.  If you do not see the word "error", there is nothing to be concerned about.)


In [63]:
import numpy as np
import pandas as pd

colleges = pd.read_csv("https://www.dropbox.com/s/bt5hvctdevhbq6j/colleges.csv?dl=1")



The code chunks in this section will cause an error and/or do the desired task incorrectly.  (Even the chunks that run without error are not correct!) You will need to find the mistake, and correct it, to complete the intended action.

None of the *text* is incorrect, only the *code* - your goal is to make the code do exactly what the documentation claims it will.

#### 1. Narrow down the columns to variables of interest

There are too many variables in this dataset.  We don't need all of them. We will limit our dataset down to only:

Name of the institution
City, State, and ZIP code of the institution
The Admissions Rate
The average SAT score
The number of undergraduate students
The in and out of state tuitions
Whether the school is public or private
The "REGION" variable.


In [64]:
colleges_clean = colleges[['INSTNM', 'CITY', 'STABBR', 'ZIP', 'CONTROL', 'ADM_RATE', 'SAT_AVG', 'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'UGDS', 'REGION']]

#### 2. Remove observations that aren't needed

Drop the schools that are "private for-profit" (category 3).  We will not need these for the puzzle.


In [65]:
colleges_clean = colleges_clean[(colleges_clean['CONTROL'] == 1)| (colleges_clean['CONTROL'] == 2)]

### 3. Adjust variable types: Numeric

Some of the columns of the dataset were automatically understood by the computer to be categorical, due to missing data appearing like strings.

Adjust the appropriate columns to be numeric.


In [66]:
colleges_clean['TUITIONFEE_IN'] = pd.to_numeric(colleges_clean['TUITIONFEE_IN'])

colleges_clean['TUITIONFEE_OUT'] = pd.to_numeric(colleges_clean['TUITIONFEE_OUT'])

colleges_clean['SAT_AVG'] = pd.to_numeric(colleges_clean['SAT_AVG'])

colleges_clean['ADM_RATE'] = pd.to_numeric(colleges_clean['ADM_RATE'])

### 4. Adjust variable types: Categorical

Similarly, columns of the dataset that are not numbers are automatically understood by the computer to be strings.

We know, however, that some variables are truly *categorical*, in that there are only a few categories (or "levels") that can be values of the column.

Adjust the appropriate columns to be understood as categorical.


In [67]:
colleges_clean['CONTROL'].astype('category')
colleges_clean['REGION'].astype('category')

0       5
1       5
2       5
3       5
4       5
       ..
7050    6
7051    6
7052    6
7053    6
7056    1
Name: REGION, Length: 4089, dtype: category
Categories (10, int64): [0, 1, 2, 3, ..., 6, 7, 8, 9]

#### 5. Calculate new variables

Create a new variable called TUITION_DIFF which contains the difference between in and out of state costs.


In [68]:
colleges_clean['TUITION_DIFF'] = colleges_clean['TUITIONFEE_OUT']- colleges_clean['TUITIONFEE_IN']

#### 6. Deal with missing data

Drop all the rows with missing data.

(It is often a terrible idea to simply drop all rows with *any* missing data!  Usually, even if *some* of the information is missing, we don't want to throw out the entire row.  In this example, however, we'll be lazy.)


In [69]:
colleges_clean = colleges_clean.dropna(axis=0)
colleges_clean

print(colleges_clean)

                                   INSTNM              CITY STABBR  ...     UGDS  REGION  TUITION_DIFF
0                Alabama A & M University            Normal     AL  ...   4824.0       5        8379.0
1     University of Alabama at Birmingham        Birmingham     AL  ...  12866.0       5       10704.0
3     University of Alabama in Huntsville        Huntsville     AL  ...   6917.0       5       11200.0
4                Alabama State University        Montgomery     AL  ...   4189.0       5        8328.0
5               The University of Alabama        Tuscaloosa     AL  ...  32387.0       5       17320.0
...                                   ...               ...    ...  ...      ...     ...           ...
6391                     Milligan College  Milligan College     TN  ...    806.0       5           0.0
6401      Emory University-Oxford College            Oxford     GA  ...    979.0       5           0.0
6424                    Husson University            Bangor     ME  ...  

## Part Two:  Identify the mystery college

Wow!  Your best friend Ephelia has been accepted to the college of her dreams!  Unfortunately, Ephelia is a very mysterious person, and she won't tell you directly which college this is.  You'll have to use her clues to figure out which school is her dream school.

Clues:

1. This college is located in Region 1.

2. This college's admission rate is in the first quartile for the region.

3. This college charges the same for in- and out-of-state tuition.

4. The average SAT score of this college is an odd number.

5. This college is NOT in New Hampshire or in the city of Boston.

6. More than 3,000 people apply to this college every year.  (Hint: Make a new variable to compute how many people apply.  Assume the size of an incoming class is 1/4 of the total undergraduate population, and remember that not everyone who *applies* is *admitted*.)

7. Ephelia is not going to an Ivy League school.

8. Of the two options remaining at this step, Ephelia will attend the cheaper one.


In [70]:
colleges_clean = colleges_clean[(colleges_clean['REGION']==1)]

,INSTNM,CITY,STABBR,ZIP,CONTROL,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,REGION,TUITION_DIFF
578,Albertus Magnus College,New Haven,CT,06511-1189,2,0.6388,930.0,31140.0,31140.0,1122.0,1,0.0
584,University of Bridgeport,Bridgeport,CT,06604-5620,2,0.5389,969.0,32250.0,32250.0,3129.0,1,0.0
585,Central Connecticut State University,New Britain,CT,6050,1,0.6676,1071.0,10225.0,22914.0,9317.0,1,12689.0
590,University of Connecticut,Storrs,CT,6269,1,0.4784,1313.0,14880.0,36948.0,18930.0,1,22068.0
600,University of Hartford,West Hartford,CT,06117-1599,2,0.8089,1140.0,38910.0,38910.0,4835.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4764,University of Connecticut-Avery Point,Groton,CT,06340-6097,1,0.9416,1111.0,12838.0,34906.0,619.0,1,22068.0
4765,University of Connecticut-Stamford,Stamford,CT,06901-2315,1,0.8232,1085.0,12848.0,34916.0,1670.0,1,22068.0
4915,Franklin W Olin College of Engineering,Needham,MA,02492-1200,2,0.1337,1522.0,51936.0,51936.0,345.0,1,0.0
5848,University of Connecticut-Hartford Campus,Hartford,CT,6103,1,0.9597,1101.0,12828.0,34896.0,1244.0,1,22068.0


In [71]:
colleges_clean.describe()

colleges_clean = colleges_clean[(colleges_clean['ADM_RATE']<=3)]



In [98]:
colleges_clean = colleges_clean[(colleges_clean['TUITION_DIFF']==0)]

colleges_clean = colleges_clean[(colleges_clean['SAT_AVG']%2!=0)]

colleges_clean =colleges_clean[(colleges_clean['STABBR']!='NH')&(colleges_clean['CITY']!='Boston')]

colleges_clean["AP_NUM"] = (colleges_clean['UGDS']/4)/colleges_clean['ADM_RATE']
colleges_clean = colleges_clean[(colleges_clean['AP_NUM'])>=3000]

colleges_clean = colleges_clean[(colleges_clean['INSTNM']!="Harvard University")]

colleges_clean.loc[colleges_clean['TUITIONFEE_IN'].idxmin()]


INSTNM            Williams College
CITY                  Williamstown
STABBR                          MA
ZIP                           1267
CONTROL                          2
ADM_RATE                    0.1458
SAT_AVG                     1493.0
TUITIONFEE_IN              53550.0
TUITIONFEE_OUT             53550.0
UGDS                        2033.0
REGION                           1
TUITION_DIFF                   0.0
AP_NUM                 3485.939643
Name: 1619, dtype: object